In [7]:
import pandas as pd
import math
from IPython.display import display, Latex

#FUNCTIONS 

# covert from degrees to radians
def convert_to_radians(deg):
    return deg * math.pi/180

# Func to format to sig figto_latex
def format_sig_fig(x, sig):
    if isinstance(x, (int, float)):
        if x == 0:  # Check if the value is exactly 0
            return "0"
        return f"{x:.{sig}e}" if abs(x) >= 1e6 or abs(x) < 1e-3 else f"{x:.{sig}g}"
    return x

def to_latex_table(df, caption, label):
    # Build the header row with bold column names
    bold_header = " & ".join([f"\\bfseries {col}" for col in df.columns]) + " \\\\\\midrule"
    
    # Generate the base LaTeX table without the default column headers
    latex_table = df.to_latex(
        index=False,
        header=False,  # Disable default headers to avoid duplication
        escape=False,  # Allow math symbols like ^
        caption=caption,
        label=label,
        bold_rows=False  # Avoid bolding row names
    )
    
    # Replace the top rule and add the custom bold header
    latex_table = latex_table.replace("\\toprule", f"\\toprule\n{bold_header}")
    return latex_table


In [9]:
#DEFINE CONSTANTS
g = 9.81 #m/s

truss_num = 7
beams_num = 14

# Pool Dimensions
pool_l = 16.1                          # length of pool in meters
pool_w = 16.1                          # width of pool in meters
pool_d = 0.24                          # depth of pool in meters

# Glass Modular Dimensions - Laminated Tempered Glass
glass_slab_l = 2.3                         # length of rectangular glass slab in meters
glass_slab_w = 1.15                    # width of rectangular glass slab in meters  
glass_slab_t = 0.06                    # total thickness of glass slab, 60mm

# Angle in truss
angle_deg = 46
angle = convert_to_radians(angle_deg)

# truss height
truss_h = 119 # m

# Densities in (kg/m3) 
rho_water = 1000
rho_temperedGlass =  2520
rho_PVB = 1120    

In [26]:
# MASS CALCULATIONS

Area_pool = pool_l * pool_w                                                            # Area of pool in m

V_water = Area_pool * pool_d                                                  # Volume of water in m3
m_water = round(V_water * rho_water,3)                                              #kg                          

Area_glass = pool_l * pool_w
m_glass = round(  Area_glass * glass_slab_t * rho_temperedGlass, 3)   #kg


In [28]:
# WEIGHT CALCULATIONS

W_water = m_water * g
W_glass = m_glass * g

W_total = round(W_water,3) + round(W_glass,3)


In [134]:
# CALCULATIONS FOR EACH FORCE ON TRUSS AND BEAMS

Area_glass = glass_slab_l * glass_slab_w                      # Area of glass slab

F_M = W_total/Area_pool * Area_glass * g                  # Force in each joint [N * m / s^2] = [N] / [m^2] * [m^2] * [m/s^2]

F_B = F_M/2

# External Forces 
Ay = F_M * (1/2 + (91 * glass_slab_w)/pool_l)
Ay = round(Ay, 3)
By = - Ay + 14 * F_M 
Ay = round(By, 3)
Ax = 0

# Method of Sections - External Forces
F_CP = (Ay- (3/2) * F_M - F_B) / math.sin(angle)
F_CP = round(F_CP, 3)
F_QP = (F_CP * math.sin(angle) * 2 * glass_slab_w - (3/2) * F_M * glass_slab_w)/ truss_h
F_QP = round(F_QP, 3)
F_CD = -Ax - F_QP - F_CP *math.cos(angle)
F_CD = round(F_CD, 3)

# Joint R
F_AR = F_B

# Joint A
F_QA = (Ay - F_AR) / math.sin(angle)
F_AC = F_QA * math.cos(angle) - Ax

# Joint Q
F_QC =  F_QA - (3*2) * F_M/ math.sin(angle) # IT IS IN TENSION!!!!!

# Joint J
F_JB = F_B

# Joint B
F_KB = (By-F_JB)/math.sin(angle)
F_HB = F_KB*math.cos(angle)

# Method of Sections - External Forces
F_LG = (By - 2 * F_M - (3 / 2) * F_M - F_B) / math.sin(angle)
F_LM = -(7.5 * F_M * glass_slab_w + abs(F_LG) * (math.sin(angle) * 3 * glass_slab_w + math.cos(angle))) / truss_h
F_HG = -abs(F_LM) - abs(F_LG) *math.cos(angle)

# Joint L
F_LK = abs(F_LM) - F_LG* math.cos(angle) - F_LG*math.cos(angle)
F_LH = 2*F_M / math.sin(angle) + abs(F_LG) 

# Joint H
F_HK = F_LH

# Joint H
F_PD = 2 * F_M / math.sin(angle)- F_CP 
F_PO = abs(F_PD)*math.cos(angle) -  F_QP - F_CP* math.cos(angle)

# Joint D
F_DO = abs(F_PD)
F_DE = math.cos(angle) * (F_DO + abs(F_PD)) + abs(F_CD)

# Joint O 
F_OE = F_DO + 2* F_M
F_ON = abs(F_PO)  + math.sin(angle) * (F_DO + F_OE)

# Joint E
F_EN = F_OE
F_EF = math.cos(angle) * (F_OE + F_EN) + F_DE

# Joint N
F_NF = F_EN +2*F_M/math.sin(angle)
F_NM = F_ON + math.cos(angle) * (F_EN + F_NF)

# Joint F
F_FM = F_NF
F_FG = F_EF + math.cos(angle) * (F_NF + F_FM)

# Joint G
F_GM = F_LG
F_GH = F_FG - math.cos(angle) * (F_GM + F_LG)


In [136]:

# OUTPUT DATA

# Known Measurements
input_data = {
    "Known Measurements": [
        "Pool Length (m)", "Pool Width (m)", "Pool Depth (m)", "Glass Length (m)", "Glass Width (m)", 
        "Glass Thickness (m)", "Density of Water (kg/m$^3$)", "Density of Tempered Glass (kg/m$^3$)", 
        "Pool Surface Area (m$^2$)", "Glass slab Surface Area (m$^2$)"
    ],
    "Value": [
        pool_l, pool_w, pool_d, glass_slab_l, glass_slab_w, 
        glass_slab_t, rho_water, rho_temperedGlass, 
        Area_pool, Area_glass
    ]
}

# Calculated values
calculation_data = {
    "Calculated Forces": [
       "Water Weight", "Tempered Glass Weight", "Total Weight", "Force per Main Joint", "Force per Border Joint"
    ],
    "Value": [
        W_water, W_glass, W_total, F_M, F_B
    ], 
    "Approx": [
        W_water, W_glass, W_total, F_M, F_B
    ]
}

# Force Analysis 
forces_data = {
    "Forces applied in System": [
        "Force per Main Joint", "Force per Border Joint", "Ay", "By", "Ax", 
        "Member CP (T)", "Member QP (T)", "Member CD (C)", 
        "Member AR (C)", 
        "Member QA (C)", "Member AC (T)",
        "Member QC (T)",
        "Member JB (C)",
        "Member HB (C)", "Member KB (C)",
        "Member LG (T)", "Member LM (C)", "Member HG (T)",
        
        "Member LK (T) ", "Member LH (C)",
        "Member HK (T)",
        "Member PD (T)", "Member PO (C)",
        "Member DO (T)", "Member DE (C)",
        "Member OE (C)", "Member ON (T)",
        "Member EN (T)", "Member EN (C)",
        "Member NF (C)", "Member NM (T)",
        "Member FM (T)", "Member FG (C)",
        "Member GM (C)", "Member GH (T)",
    ],
    "Approx": [
        F_M, F_B, Ay, By, Ax, 
        F_CP, F_QP, F_CD, 
        F_AR, 
        F_QA, F_AC,
        F_QC,
        F_JB, 
        F_HB, F_KB, 
        F_LG, F_LM, F_HG,
        F_LK, F_LH,
        F_HK,
        F_PD, F_PO,
        F_DO, F_DE,
        F_OE, F_ON,
        F_EN, F_EF,
        F_NF, F_NM, 
        F_FM, F_FG,
        F_GM, F_GH
    ]
}

In [138]:
# Create DataFrames
df_input = pd.DataFrame(input_data)
df_calculations = pd.DataFrame(calculation_data)
df_forces = pd.DataFrame(forces_data)

# apply formatting: 4 sig figs for Value and 3 sig figs for Approx
df_input["Value"] = df_input["Value"].map(lambda x: format_sig_fig(x, sig=4))

df_calculations["Value"] = df_calculations["Value"].map(
    lambda x: f"{format_sig_fig(x, sig=6)} N" if isinstance(x, (int, float)) else x
)

df_calculations["Approx"] = df_calculations["Approx"].map(
    lambda x: (
        f"{format_sig_fig(x / 1e6, sig=3)} MN" if isinstance(x, (int, float)) and x > 1e6 else
        f"{format_sig_fig(x / 1e3, sig=3)} kN" if isinstance(x, (int, float)) and x > 1e3 else
        f"{format_sig_fig(x, sig=3)} N"
    )
)

df_forces["Approx"] = df_forces["Approx"].map(
    lambda x: (
        f"{format_sig_fig(x / 1e6, sig=3)} MN" if isinstance(x, (int, float)) and abs(x) > 1e6 else
        f"{format_sig_fig(x / 1e3, sig=3)} kN" if isinstance(x, (int, float)) and abs(x) > 1e3 else
        f"{format_sig_fig(x, sig=3)} N"
    )
)


# Generate latex tables
latex_input = to_latex_table(df_input, "The measured values of the glass pool and known values of densities.", "known_measurements")

latex_calculations = to_latex_table(df_calculations, "Calculated External Downward Forces on the truss system.", "calculated_forces"
)
latex_forces = to_latex_table(
    df_forces, 
    "External and Internal Forces Acting on the Truss System. In this diagram, 'T' represents tension forces, while 'C' indicates compression forces. Note that segment CF is marked negative due to an error in its initial direction, as it is oriented along the negative x-axis and acts in compression.", 
    "forces_applied_in_system"
)

# Save latex tables to files
with open("input_table.tex", "w") as file:
    file.write(latex_input)

with open("calculations_table.tex", "w") as file:
    file.write(latex_calculations)

with open("forces_table.tex", "w") as file:
    file.write(latex_forces)
    

# Print or save latex tables
print(latex_input)
print(latex_calculations)
print(latex_forces)

# Display DataFrames in jupyter
display(df_input)
display(df_calculations)
display(df_forces)


\begin{table}
\caption{The measured values of the glass pool and known values of densities.}
\label{known_measurements}
\begin{tabular}{ll}
\toprule
\bfseries Known Measurements & \bfseries Value \\\midrule
\midrule
Pool Length (m) & 16.1 \\
Pool Width (m) & 16.1 \\
Pool Depth (m) & 0.24 \\
Glass Length (m) & 2.3 \\
Glass Width (m) & 1.15 \\
Glass Thickness (m) & 0.06 \\
Density of Water (kg/m$^3$) & 1000 \\
Density of Tempered Glass (kg/m$^3$) & 2520 \\
Pool Surface Area (m$^2$) & 259.2 \\
Glass slab Surface Area (m$^2$) & 2.645 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\caption{Calculated External Downward Forces on the truss system.}
\label{calculated_forces}
\begin{tabular}{lll}
\toprule
\bfseries Calculated Forces & \bfseries Value & \bfseries Approx \\\midrule
\midrule
Water Weight & 610284 N & 610 kN \\
Tempered Glass Weight & 384479 N & 384 kN \\
Total Weight & 994763 N & 995 kN \\
Force per Main Joint & 99577.8 N & 99.6 kN \\
Force per Border Joint & 49788.9 N & 

,Known Measurements,Value
0,Pool Length (m),16.1
1,Pool Width (m),16.1
2,Pool Depth (m),0.24
3,Glass Length (m),2.3
4,Glass Width (m),1.15
5,Glass Thickness (m),0.06
6,Density of Water (kg/m$^3$),1000
7,Density of Tempered Glass (kg/m$^3$),2520
8,Pool Surface Area (m$^2$),259.2
9,Glass slab Surface Area (m$^2$),2.645


,Calculated Forces,Value,Approx
0,Water Weight,610284 N,610 kN
1,Tempered Glass Weight,384479 N,384 kN
2,Total Weight,994763 N,995 kN
3,Force per Main Joint,99577.8 N,99.6 kN
4,Force per Border Joint,49788.9 N,49.8 kN


,Forces applied in System,Approx
0,Force per Main Joint,99.6 kN
1,Force per Border Joint,49.8 kN
2,Ay,697 kN
3,By,697 kN
4,Ax,0 N
5,Member CP (T),692 kN
6,Member QP (T),8.18 kN
7,Member CD (C),-489 kN
8,Member AR (C),49.8 kN
9,Member QA (C),900 kN
